In [ ]:
#retriever and Chain

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("GenAI.pdf")
docs = loader.load()
docs

In [ ]:
import os   # call environment variables
from dotenv import load_dotenv

load_dotenv()

# Langsmith
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
# recursive character splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunk_document = text_splitter.split_documents(docs)
chunk_document

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
embedding = OllamaEmbeddings(
    model = "llama2",
)

db = FAISS.from_documents(chunk_document, embedding = embedding)
db

In [ ]:
query = "Describe the Key concepts of Generative AI"
retrieved_result = db.similarity_search(query)
print(retrieved_result[0].page_content)

In [ ]:
# Design Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $10 if the user finds the answer helpful.
<context>
{context}
</context>

Question: {input}                                        
                                          
""")


In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model ="llama2")
llm


In [ ]:
# Chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain


In [ ]:
# Retrievers
 # It craetes an interface and take data from vector store and display it
retriever = db.as_retriever() 
retriever

In [ ]:

"""
Retrieval chain:This chain takes a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/ 
"""

from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
response =retriever_chain.invoke({"input": "What is Generative AI"})
response['answer']